In [1]:
from get_biblatex import GetBiblatex
import pandas as pd


## SS-ids of Staff members

In [51]:
ss_data = pd.read_csv(r"C:\Users\drepeeters\OneDrive - Radboudumc\Desktop\webteam\Literature\scripts\script_data\temp_semantic_scholar_all_staff_id_last_500_summary.csv")
ss_ids = ss_data['ss_id'].unique().tolist()
len(ss_ids)

1611

## Double Diag bibkey

In [3]:
multiple_data = pd.read_csv(r"C:\Users\drepeeters\OneDrive - Radboudumc\Desktop\webteam\Literature\scripts\script_data\TEMP_LOOKUP_multiple_bibkeys_maybe_multiple_ss_ids.csv")
multiple_data = multiple_data.dropna(subset=["ss_id"])
multiple_ss_ids = multiple_data['ss_id'].unique().tolist()
len(multiple_ss_ids)

37

## Single diag bibkeys

In [52]:
lookup_data = pd.read_csv(r"C:\Users\drepeeters\OneDrive - Radboudumc\Desktop\webteam\Literature\scripts\script_data\TEMP_LOOKUP_matches_single_bibkey_with_none_one_or_many_ss_ids.csv")
lookup_data = lookup_data.dropna(subset=["ss_id"])
lookup_ss_ids = lookup_data['ss_id'].unique().tolist()
len(lookup_ss_ids)

1017

## Get doi's of SS_ids that are not in TEMP_LOOKUP

In [53]:
new_ids = [new_id for new_id in ss_ids if new_id not in lookup_ss_ids]
len(new_ids)

594

In [54]:
dois = ss_data[ss_data['ss_id'].isin(new_ids)].dropna(subset=["doi"])["doi"].unique().tolist()
print(f"Total number of unique doi's {len(dois)}")

Total number of unique doi's 440


In [55]:
with open(r"C:\Users\drepeeters\OneDrive - Radboudumc\Desktop\webteam\Literature\diag.bib", encoding="utf8") as bibtex_file:
    diag_bib = bibtex_file.read()

In [ ]:
for index, doi in enumerate(dois):
    print(f"Working on {index}/{len(dois)}: {doi}")
    try:
        reader = GetBiblatex(doi=doi, diag_bib=diag_bib)
        biblatex = reader.get_bib_text()
    except Exception as e:
        print(e)



In [56]:
authors, titles, years, dois_to_check = [], [], [], []
for index2, doi in enumerate(dois):
    print(f"Working on {index2}/{len(dois)}: {doi}")
    reader = GetBiblatex(doi=doi, diag_bib=diag_bib)
    response = reader._get_doi_csl()

    try:
        author = response['author']
        author_string = "{"
        for index, author in enumerate(author):
            if index == len(author)-1:
                if 'given' in author:
                    author_string = author_string + f"{author['family']}, {author['given']}" + "}"
                else:
                    author_string = author_string + f"{author['family']} and "
            else:
                if 'given' in author:
                    author_string = author_string + f"{author['family']}, {author['given']} and "
                else:
                    author_string = author_string + f"{author['family']} and "

        title = response["title"]
        year = str(response["published"]["date-parts"][0][0])
        dois_to_check.append(doi)
        authors.append(author_string)
        titles.append(title)
        years.append(year)
    except Exception as e:
        print(e)
        author = 'not available'
        title = 'not available'
        year = 'not available'


Working on 0/440: 10.1016/j.media.2022.102605
Working on 1/440: 10.1177/0271678X18756218
Working on 2/440: 10.1007/978-3-642-59327-7_37
Working on 3/440: 10.1007/978-3-540-39701-4_33
Working on 4/440: 10.1016/J.OFTAL.2011.04.015
Working on 5/440: 10.1055/B-0034-84660
'author'
Working on 6/440: 10.1055/B-0034-84648
'author'
Working on 7/440: 10.1074/jbc.M710314200
Working on 8/440: 10.1023/B:NEON.0000013493.31107.20
Working on 9/440: 10.1038/s41585-020-0324-x
'family'
Working on 10/440: 10.1148/radiol.2019182666
Working on 11/440: 10.1007/978-3-319-07887-8_80
Working on 12/440: 10.1016/j.ejso.2020.04.038
Working on 13/440: 10.23698/AIDA/BRLN
'published'
Working on 14/440: 10.1002/path.2613
Working on 15/440: 10.1007/978-3-642-59327-7_100
Working on 16/440: 10.1055/B-0034-84659
'author'
Working on 17/440: 10.1093/neuonc/now292.004
Working on 18/440: 10.1016/j.oraloncology.2013.09.006
Working on 19/440: 10.1002/hed.10100
Working on 20/440: 10.1513/ANNALSATS.201809-601OC
Working on 21/440:

In [57]:
data = pd.DataFrame(list(zip(dois_to_check, authors, titles, years)), columns=['doi', 'authors', 'title', 'year'])

In [60]:
data.to_csv("script_data/temp_overview_new_dois.csv", index=False)

In [61]:
len(data)

349